### Importing packages

In [18]:
import gym
import option_price_process
import HedgeEnv_PPO
import HedgeEnv_DQN
import numpy as np
import torch
import matplotlib.pyplot as plt
import tensorflow as tf
from asset_price_process import GBM
from option_price_process import BSM
from HedgeEnv import env_hedging
from torch import nn
import time


### Setting Up Environment, Asset Price model, and Option Pricing Model

In [ ]:
mu = 0
dt = 1/5
T = 10
num_steps = int(T/dt)
s_0 = float(100)
strike_price = s_0
sigma = 0.1
r = 0

def cost(delta_h, multiplier):
    TickSize = 0.1
    return multiplier * TickSize * (np.abs(delta_h) + 0.01 * delta_h**2)


apm = GBM(mu=mu, dt=dt, s_0=s_0, sigma=sigma)
opm = BSM(strike_price=strike_price, risk_free_interest_rate=r, volatility=sigma, T=T, dt=dt)
env = HedgeEnv_DQN.env_hedging(asset_price_model=apm, dt=dt, T=T, num_steps=num_steps, cost_multiplier = 0, tick_size=0.01,
                     L=1, strike_price=strike_price, int_holdings=True, initial_holding=0, mode="PL",
                  option_price_model=opm)




### Untrained Agent making random actions

We simulate a BSM world, but modified to renact the realities of trading: Discrete time
and space. We consider a stock whose price process is a geometric Brownian motion (GBM)
with initial price $S_{0}$ and daily lognormal volatility of $\sigma$/day

### DQN Agent using StableBaselines 

In [ ]:
from stable_baselines3 import DQN
from stable_baselines3.common.torch_layers import BaseFeaturesExtractor
from stable_baselines3.dqn.policies import DQNPolicy
from stable_baselines3.common.utils import get_schedule_fn
import os

env = HedgeEnv_DQN.env_hedging(asset_price_model=apm, dt=dt, T=T, num_steps=num_steps, cost_multiplier = 0, tick_size=0.01,
                     L=1, strike_price=strike_price, int_holdings=True, initial_holding=0, mode="PL",
                  option_price_model=opm)

# Custom Feature Extractor with LSTM
class CustomFeatureExtractor(BaseFeaturesExtractor):
    def __init__(self, observation_space: gym.spaces.Box, features_dim: int = 201):
        super(CustomFeatureExtractor, self).__init__(observation_space, features_dim)
        
        # Assuming the input is a 1D vector, reshape it to (batch_size, sequence_length, features)
        input_dim = observation_space.shape[0]

        self.lstm = nn.LSTM(input_dim, 128, batch_first=True)
        self.fc_net = nn.Sequential(
            nn.Linear(128, 256),
            nn.ReLU(),
            nn.Linear(256, features_dim),
            nn.ReLU(),
        )

    def forward(self, observations: torch.Tensor) -> torch.Tensor:
        # Reshape the input to add a sequence length of 1
        observations = observations.unsqueeze(1)
        lstm_out, _ = self.lstm(observations)
        # Use the output of the last time step
        last_timestep_output = lstm_out[:, -1, :]
        return self.fc_net(last_timestep_output)

# Custom DQN Policy
class CustomDQNPolicy(DQNPolicy):
    def __init__(
        self,
        observation_space: gym.spaces.Space,
        action_space: gym.spaces.Discrete,
        lr_schedule,
        net_arch=None,
        activation_fn=nn.ReLU,
        features_extractor_class=CustomFeatureExtractor,
        features_extractor_kwargs=None,
        normalize_images=False,
        optimizer_class=torch.optim.Adam,
        optimizer_kwargs=None,
    ):
        super(CustomDQNPolicy, self).__init__(
            observation_space,
            action_space,
            lr_schedule,
            net_arch=net_arch,
            activation_fn=activation_fn,
            features_extractor_class=features_extractor_class,
            features_extractor_kwargs=features_extractor_kwargs,
            normalize_images=normalize_images,
            optimizer_class=optimizer_class,
            optimizer_kwargs=optimizer_kwargs,
        )

# Learning rate schedule
    
lr_schedule = get_schedule_fn(1e-4)

from stable_baselines3.common.callbacks import BaseCallback

class RewardCallback(BaseCallback):
    def __init__(self, check_freq: int, verbose=1):
        super(RewardCallback, self).__init__(verbose)
        self.check_freq = check_freq
        self.rewards = []

    def _on_step(self) -> bool:
        # Save the reward for this step
        reward = self.locals["rewards"][0]  # Access the current reward
        if self.n_calls % self.check_freq == 0:
            self.rewards.append(reward)  # Log rewards at each check frequency
        return True

# Usage
callback = RewardCallback(check_freq=1000, verbose=1)

models_dir = "models/DQN"
logdir = "logs"

if not os.path.exists(models_dir):
    os.makedirs(models_dir)

if not os.path.exists(logdir):
    os.makedirs(logdir)


model_DQN = DQN(policy= CustomDQNPolicy, 
    env=env,
    learning_rate=lr_schedule,
    verbose=1,
    gamma= 0.85, 
    batch_size=32,
    tensorboard_log= logdir)

TIMESTEPS = 3000000
model_DQN.learn(total_timesteps= TIMESTEPS, reset_num_timesteps= False, tb_log_name="DQN", callback= callback)
model_DQN.save(f"{models_dir}/{TIMESTEPS}")





# num_episodes = 100

# for episode in range(num_episodes): 
#     state = env.reset()
#     done = False
#     cum_option_pnl = 0
#     cum_stock_pnl = 0
#     cum_cost_pnl = 0
#     option_pnls = [0]
#     stock_pnls = [0]
#     cost_pnls = [0]
#     total_pnl = [0]
#     stock_pos_shares =[state[0]]
#     delta_pos_shares = [-100*round(state[4], 2)]
    
#     while not done:
#         current_state = env.get_state()
#         delta_action = -100 * round(current_state[4], 2)
#         action, _states = model.predict(current_state, deterministic = True)
#         next_state, reward, done, info = env.step(action-0)
#         delta_h =   next_state[0] -current_state[0]
#         option_pnl = (100*(next_state[3]-current_state[3]))/100
#         stock_pnl = (-next_state[0]*(next_state[1] -current_state[1])- cost(delta_h, 5))/100
#         cum_option_pnl += option_pnl
#         cum_stock_pnl += stock_pnl
#         cum_cost_pnl += cost(delta_h, 5)/100

#         option_pnls.append(cum_option_pnl)
#         stock_pnls.append(cum_stock_pnl)
#         cost_pnls.append(cum_cost_pnl)
#         total_pnl.append(cum_option_pnl+cum_stock_pnl)
#         stock_pos_shares.append(next_state[0])
#         delta_pos_shares.append(delta_action)

#         if done:
#             state = env.reset()

#     time_axis = np.linspace(0, num_steps, num_steps + 1)
#     plt.figure(figsize=(12, 8))
#     plt.plot(time_axis, stock_pnls, label='Stock P&L', color='green', linestyle='-')
#     plt.plot(time_axis, option_pnls, label='Option P&L', color='red', linestyle='--')
#     plt.plot(time_axis, cost_pnls, label='Cost P&L', color='magenta', linestyle='--')
#     plt.plot(time_axis, total_pnl, label='Total P&L', color='black', linestyle='-.')
#     plt.plot(time_axis, stock_pos_shares, label='Stock Position Shares', color='blue', linestyle=':')
#     plt.plot(time_axis, delta_pos_shares, label='Delta Position Shares', color='orange', linestyle='--')

#     # Add labels, title, and legend
#     plt.xlabel('Timestep (D * T)', fontsize=14)
#     plt.ylabel('Value (Dollars or Shares)', fontsize=14)
#     plt.title('Cumulative P&L and Positions Over Time', fontsize=16)
#     plt.legend(fontsize=12)
#     plt.grid(True)
#     plt.savefig("Figure2_Baseline")

#     plt.show()

In [ ]:
from stable_baselines3 import DQN
from stable_baselines3.common.torch_layers import BaseFeaturesExtractor
from stable_baselines3.dqn.policies import DQNPolicy
from stable_baselines3.common.utils import get_schedule_fn
from stable_baselines3.common.callbacks import EvalCallback, CheckpointCallback
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.evaluation import evaluate_policy

model = DQN.load("dqn_hedge")

evaluate_policy(model, env, n_eval_episodes= 1000)

In [ ]:
def student_t_statistic(data):
    n = len(data)
    sample_mean = np.mean(data)
    sample_std = np.std(data, ddof=1)
    t_statistic = sample_mean / (sample_std / np.sqrt(n))
    return t_statistic

# Gettting kernel density estimates for cost and volatility 
num_episodes = 1
cost_pnls_dh= []
cost_pnls_reinf = []
total_pnls_vol_dh =[]
total_pnls_vol_reinf = []

for episode in range(num_episodes): 
    state = env.reset()
    done = False
    dh_actions = [0]
    cum_cost_pnl_dh = 0
    cum_cost_pnl_reinf = 0
    cum_total_pnl_dh = []
    cum_total_pnl_reinf = [] 
    deltas =[]
    
    while not done:
        current_state = env.get_state()
        delta_action = -100 * round(current_state[4], 2)
        deltas.append(delta_action)
        delta_h_dh= delta_action - dh_actions[-1]
        dh_actions.append(delta_action)

        action, _states = model.predict(current_state, deterministic = False)
        next_state, reward, done, info = env.step(action-0)
        delta_h_reinf =   next_state[0] -current_state[0]
        

        cum_cost_pnl_dh += cost(delta_h_dh, 5)
        cum_cost_pnl_reinf += cost(delta_h_reinf, 5)

        total_pnl_reinf = ((100*(next_state[3]-current_state[3])) + (-next_state[0]*(next_state[1] -current_state[1])- cost(delta_h_reinf, 5)))/100
        total_pnl_dh = ((100*(next_state[3]-current_state[3])) + (-delta_action*(next_state[1] -current_state[1])- cost(delta_h_dh, 5)))/100
        cum_total_pnl_reinf.append(total_pnl_reinf)
        cum_total_pnl_dh.append(total_pnl_dh)
        
        if done:
            state = env.reset()

    vol_reinf = np.std(cum_total_pnl_reinf)
    vol_dh = np.std(cum_total_pnl_dh)

    cost_pnls_reinf.append(cum_cost_pnl_reinf)
    cost_pnls_dh.append(cum_cost_pnl_dh)
    total_pnls_vol_reinf.append(vol_reinf)
    total_pnls_vol_dh.append(vol_dh)


In [ ]:
from scipy import stats
import seaborn as sns

# Plot kernel density estimates for total cost
plt.figure(figsize=(14, 6))

plt.subplot(1, 2, 1)
sns.kdeplot(cost_pnls_dh, label='Policy: $\delta_{DH}$', shade=True)
sns.kdeplot(cost_pnls_reinf, label='Policy: reinf', shade=True)
plt.title('KDE for Total Cost')
plt.xlabel('Total Cost')
plt.ylabel('Density')
plt.legend()

# Plot kernel density estimates for volatility of total P&L
plt.subplot(1, 2, 2)
sns.kdeplot(total_pnls_vol_dh, label='Policy: $\delta_{DH}$', shade=True)
sns.kdeplot(total_pnls_vol_reinf, label='Policy: reinf', shade=True)
plt.title('KDE for Volatility of Total P&L')
plt.xlabel('Volatility of Total P&L')
plt.ylabel('Density')
plt.legend()

plt.tight_layout()

plt.savefig("Figure5_KernelDensities")
plt.show()


### Building a PPO Agent 

# Build the Deep Hedging Agent

In [ ]:
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Dense, Flatten
# from tensorflow.keras.optimizers import Adam
# from rl.agents import DQNAgent
# from rl.policy import BoltzmannQPolicy
# from rl.memory import SequentialMemory

In [ ]:
# def build_agent(model, actions):
#     policy = BoltzmannQPolicy()
#     memory = SequentialMemory(limit=50000, window_length=1)
#     dqn = DQNAgent(model=model, memory=memory, policy=policy, 
#                   nb_actions=actions, nb_steps_warmup=10, target_model_update=1e-2)
#     return dqn



In [ ]:
# dqn = build_agent(model, actions)
# dqn.compile(Adam(lr=1e-3), metrics=['mae'])
# dqn.fit(env, nb_steps=50000, visualize=False, verbose=1)

In [ ]:
# class QNet(nn.Module):
#     def __init__(self, state_dim, hidden_dim, action_num):
#         super(QNet, self).__init__()

#         self.fc1 = nn.Linear(state_dim, hidden_dim)
#         self.fc2 = nn.Linear(hidden_dim, hidden_dim)
#         self.fc3 = nn.Linear(hidden_dim, action_num)

#     def forward(self, state):
#         a = torch.relu(self.fc1(state))
#         a = torch.relu(self.fc2(a))
#         return self.fc3(a)

# qnet = QNet(4, 20, num_actions)
# qnet_t = QNet(4, 20, num_actions)

# dqn = DQN(qnet, qnet_t,
#           torch.optim.Adam,
#           nn.MSELoss(reduction='sum'), discount=0.8, epsilon_decay=0.999, learning_rate=0.001,
#           lr_scheduler=torch.optim.lr_scheduler.StepLR, lr_scheduler_kwargs=[{"step_size": 1000*128}])

# num_eps = 5000
# norm_factor = 10000000


# def test_delta(n=10):
#     rew = []
#     for i in range(n):
#         state = env.reset()
#         done = False
#         state = state[[0, 1, 2, 4]]
#         while not done:
#             action = state[3] - env.h
#             new_state, reward, done = env.step(action)
#             reward = np.sum(reward)
#             new_state = new_state[[0, 1, 2, 4]]
#             reward = -(reward) ** 2 + 1 / 1000 * reward
#             #reward = -(action + env.h - state[3]) ** 2  # remove that
#             rew.append(reward)
#             state = new_state
#     return np.mean(rew)


# def test(n=10):
#     rew = []
#     for i in range(n):
#         state = env.reset()
#         done = False
#         state = state[[0, 1, 2, 4]]
#         while not done:
#             out = dqn.act_discrete({"state": torch.tensor(state, dtype=torch.float32).unsqueeze(0)})
#             action = out.squeeze().detach().numpy() / num_actions
#             new_state, reward, done = env.step(action - env.h)
#             reward = np.sum(reward)
#             new_state = new_state[[0, 1, 2, 4]]
#             reward = -(reward) ** 2 + 1 / 1000 * reward
#             #reward = -(action + env.h - state[3]) ** 2  # remove that
#             if i == 1:
#                 print(action, state[3])
#             rew.append(reward)
#             state = new_state
#     return np.mean(rew)

# rew = []

# for j in range(num_eps):
#     print("episode: ", j)
#     state = env.reset()
#     done = False
#     state = state[[0,1,2,4]]
#     while not done:
#         out = dqn.act_discrete_with_noise({"state": torch.tensor(state, dtype=torch.float32).unsqueeze(0)})
#         action = out.squeeze().detach().numpy()/num_actions - env.h
#         new_state, reward, done = env.step(action)
#         #print(action)
#         #print(reward)
#         reward = np.sum(reward)
#         #print(reward)
#         new_state = new_state[[0, 1, 2, 4]]
#         #print(state)
#         reward = -norm_factor*((reward) ** 2 + 1 / 1000 * reward)
#         rew.append(reward)

#         dqn.store_transition({
#             "state": {"state": torch.tensor(state, dtype=torch.float32).unsqueeze(0)},
#             "action": {"action": out},
#             "next_state": {"state": torch.tensor(new_state, dtype=torch.float32).unsqueeze(0)},
#             "reward": float(reward),  # norm factor
#             "terminal": done
#         })
#         state = new_state

#     if j % 50 == 0 and j != 0:
#         print(test(10), test_delta(10))
#         print("reward: ", np.mean(rew), np.mean(rew)/norm_factor)
#         rew = []

#     if j > 100:
#         for _ in range(int(num_steps)):
#             dqn.update()

# #dqn.save("dqn_model_1000")

# rew_m_l = []
# cost_m_l = []
# for j in range(100):
#     rew = []
#     cost_l = []
#     state = env.reset()
#     done = False
#     state = state[[0,1,2,4]]
#     while not done:
#         out = dqn.act_discrete({"state": torch.tensor(state, dtype=torch.float32).unsqueeze(0)})
#         action = out.squeeze().detach().numpy()/num_actions - env.h
#         new_state, reward, done = env.step(action)
#         cost = reward[1]
#         reward = np.sum(reward)

#         new_state = new_state[[0, 1, 2, 4]]
#         rew.append(reward)
#         cost_l.append(cost)
#         state = new_state
#     rew_m_l.append(np.sum(rew))
#     cost_m_l.append(np.sum(cost_l))
# print("__________")
# print(rew_m_l)
# print(np.mean(rew_m_l), np.std(rew_m_l))
# print("__________")
# print(cost_m_l)
# print(np.mean(cost_m_l), np.std(cost_m_l))

# rew_m_l = []
# cost_m_l = []
# for j in range(100):
#     rew = []
#     cost_l = []
#     state = env.reset()
#     done = False
#     state = state[[0, 1, 2, 4]]
#     while not done:
#         action = state[3] - env.h
#         new_state, reward, done = env.step(action)
#         cost = reward[1]
#         reward = np.sum(reward)

#         new_state = new_state[[0, 1, 2, 4]]
#         rew.append(reward)
#         cost_l.append(cost)
#         state = new_state
#     rew_m_l.append(np.sum(rew))
#     cost_m_l.append(np.sum(cost_l))
# print("__________")
# print(rew_m_l)
# print(np.mean(rew_m_l), np.std(rew_m_l))
# print("__________")
# print(cost_m_l)
# print(np.mean(cost_m_l), np.std(cost_m_l))
# print("__________")
# print(tim - time.time())